In [1]:
import os

In [2]:
list_file = list(os.walk('./'))[0][2]
if '.DS_Store' in list_file:
    list_file.remove('.DS_Store')
if '0.null.txt' in list_file:
    list_file.remove('0.null.txt')
    
for file in list_file:
    if '替代料总表' in file:
        alternate = './' + file

print('alternate:   ' + alternate)

alternate:   ./替代料总表03-12.xlsx


In [3]:
import pandas as pd

In [4]:
df_alternate = pd.read_excel(alternate,
                             header=1,
                             dtype=str).fillna('')

df_alternate

,Unnamed: 0,NO.,美东总库存,美西总库存,美南总库存,美中库存,主料SKU,主料美东库存,主料美西库存,主料美南库存,...,库存.2,SKU3,库存.3,SKU4,库存.4,SKU5,库存.5,SKU6,库存.6,总库存
0,,1,0,0,0,0,USCA171876D-3,0,0,0,...,0,USCAB3376D,0,,,,,,,0
1,,2,0,0,0,0,USWCP52105-2,0,0,0,...,0,,,,,,,,,0
2,,3,0,0,0,0,USHM91290,0,0,0,...,,,,,,,,,,0
3,,4,0,0,0,0,USWL02211H,0,0,0,...,,,,,,,,,,0
4,,5,0,0,0,0,USFB86G00,0,0,0,...,,,,,,,,,,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69730,,69731,0,0,0,0,USFFT0M20B0,0,0,0,...,,,,,,,,,,0
69731,,69732,0,0,0,0,USPCM0FO36-C,0,0,0,...,,,,,,,,,,0
69732,,69733,0,0,0,0,USPCM0FO51-C,0,0,0,...,,,,,,,,,,0
69733,,69734,0,0,0,0,USPCM0FO40-C,0,0,0,...,,,,,,,,,,0


In [5]:
list_column = df_alternate.columns.tolist()
list_column = [column for column in list_column if '料' in column and '库存' not in column]

list_column

['主料SKU', '替换料1', '替换料2', '替换料3', '替换料4', '替换料5', '替换料6']

In [6]:
df_alternate = df_alternate[list_column]
df_alternate.columns = [column for column in range(len(list_column))]

df_alternate

,0,1,2,3,4,5,6
0,USCA171876D-3,USCA171876-3,,,,,
1,USWCP52105-2,,,,,,
2,USHM91290,,,,,,
3,USWL02211H,USWL02211,,,,,
4,USFB86G00,,,,,,
...,...,...,...,...,...,...,...
69730,USFFT0M20B0,,,,,,
69731,USPCM0FO36-C,,,,,,
69732,USPCM0FO51-C,,,,,,
69733,USPCM0FO40-C,,,,,,


In [8]:
def recursive(input_):
    # 1. 加row
    input_['row'] = [str(i+1) for i in range(len(input_))]
    
    # 2. 变形
    list_df = []
    for column in input_.columns.tolist()[:-1]:
        df_temp =  input_[[column, 'row']]
        df_temp.columns = ['column', 'row']
        df_temp = df_temp[df_temp['column'] != '']
        list_df.append(df_temp)
    df_temp_1 = pd.concat(list_df, ignore_index=True).drop_duplicates().reset_index(drop=True)
    
    # 3. 按column聚合
    df_temp_2 = df_temp_1.groupby(['column'], as_index=False).agg({'row': ';'.join})
    
    # 4. 筛选
    df_temp_2 = df_temp_2[df_temp_2['row'].str.contains(';')].reset_index(drop=True)
    df_temp_2['row'] = [';'.join(sorted(row.split(';'))) for row in df_temp_2['row'].tolist()]
    df_temp_2 = df_temp_2.drop_duplicates(['row'], keep='first').reset_index(drop=True)
    
    if len(df_temp_2) == 0:
        df_temp_2 = df_temp_1.groupby(['row'], as_index=False).agg({'column': ';'.join})
        df_temp_2 = df_temp_2['column'].str.split(';', expand=True).fillna('')
        df_temp_2.columns = [str(i) for i in range(len(df_temp_2.columns))]

        list_df = []
        for column in df_temp_2.columns.tolist()[1:]:
            df_temp =  df_temp_2[['0', column]]
            df_temp.columns = ['A', 'B']
            df_temp = df_temp[df_temp['B'] != '']
            list_df.append(df_temp)
        df_temp_2 = pd.concat(list_df, ignore_index=True).drop_duplicates().reset_index(drop=True)
        
        return df_temp_2
    
    # 5. 分列
    df_temp_2 = df_temp_2['row'].str.split(';', expand=True).fillna('')
    
    df_temp_2 = recursive(df_temp_2)
    
    df_temp_2 = pd.merge(left=df_temp_1,
                         right=df_temp_2,
                         how='left',
                         left_on='row',
                         right_on='B').fillna('')
    df_temp_2['row'] = [row if a == '' else a for row, a in zip(df_temp_2['row'].tolist(), df_temp_2['A'].tolist())]
    df_temp_2 = df_temp_2[['column', 'row']]
    df_temp_2 = df_temp_2.drop_duplicates(ignore_index=True)

    df_temp_2 = df_temp_2.groupby(['row'], as_index=False).agg({'column': ';'.join})
    df_temp_2 = df_temp_2['column'].str.split(';', expand=True).fillna('')
    df_temp_2.columns = [str(i) for i in range(len(df_temp_2.columns))]

    list_df = []
    for column in df_temp_2.columns.tolist()[1:]:
        df_temp =  df_temp_2[['0', column]]
        df_temp.columns = ['A', 'B']
        df_temp = df_temp[df_temp['B'] != '']
        list_df.append(df_temp)
    df_temp_2 = pd.concat(list_df, ignore_index=True).drop_duplicates().reset_index(drop=True)
    
    return df_temp_2

In [9]:
df_alternate = recursive(df_alternate)

df_alternate

/var/folders/20/g09tvx4s7mdd977ps185l3_r0000gn/T/ipykernel_77546/1604884347.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  input_['row'] = [str(i+1) for i in range(len(input_))]


,A,B
0,USCA171876D-3,USCA171876-3
1,USSA335083LR-C,USSA335083LR
2,USSA341349-2-C,USSA341349-2
3,USSA341150LR-C,USSA341150LR
4,USSA344469-2-C,USSA344469-2
...,...,...
56989,USGS6310TNZB-C,USGS6310TNZ
56990,USIC2A366Z-C,USIC2A366
56991,USIC71643I-C,USIC71643Z
56992,USIC02680L-6-C,USIC02680-6


In [ ]:
import datetime

In [ ]:
df_alternate = df_alternate.groupby(['A'], as_index=False).agg({'B': ';'.join})
new_df_alternate = df_alternate['B'].str.split(';', expand=True).fillna('')
new_df_alternate.columns = ['替换料' + str(i+1) for i in range(len(new_df_alternate.columns))]
new_df_alternate['主料SKU'] = df_alternate['A']
new_df_alternate = new_df_alternate[[new_df_alternate.columns.tolist()[-1]] + new_df_alternate.columns.tolist()[:-1]]

new_df_alternate.to_excel('./整理好的替代料总表_' + datetime.datetime.now().strftime("%Y%m%d_%H%M%S") + '.xlsx', index=False)

new_df_alternate